In [13]:
# Import libraries
import os
import wget
import folium
import pandas as pd
from ast import literal_eval

# Pick the right color depending on availability
def plot_color_cars_remaining(ncars):
    if (ncars < 2):
        colorcode=ncars
    else:
        colorcode=2
    return colorcode

# Color map to indicate auto availability
colorMapa = ["#555555", # 0 cars
             "#008000", # 1 car
             "#00ff00"] # > 1 car


# List of the available free tiles defaults
tiles_style = ["OpenStreetMap",
               "Stamen Terrain", 
               "Stamen Toner", 
               "Stamen Watercolor", 
               "CartoDB positron", 
               "CartoDB dark_matter"]

# input file auto
# it used to be updated in real-time, but does not seem to be anymore
infpa = "./data/autolib-disponibilite-temps-reel.csv"
if not os.path.isfile(infpa):
   wget.download('https://opendata.paris.fr/explore/dataset/autolib-disponibilite-temps-reel/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true',
              infpa)

# input file parking
infpp = "./data/parcs-de-stationnement-concedes-de-la-ville-de-paris.csv"
if not os.path.isfile(infpp):
    wget.download('https://opendata.paris.fr/explore/dataset/parcs-de-stationnement-concedes-de-la-ville-de-paris/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true',
              infpp)

# input file gas stations
infpc = "./data/pdv.csv"
if not os.path.isfile(infpc):
    wget.download('https://www.data.gouv.fr/fr/datasets/r/39d661e8-b990-4a56-909a-1fb25fd7f411', infpc)

# input file taxi stations of the Paris Taxis mobile app
infpt = "./data/paris_taxis_stations.csv"
if not os.path.isfile(infpt):
    # get taxi file
    wget.download('https://opendata.paris.fr/explore/dataset/paris_taxis_stations/download/?format=csv&timezone=Europe/Berlin&use_labels_for_header=true',
              infpt)

# Center the map in Paris
m = folium.Map(location=[48.859553, 2.336332], 
               tiles=tiles_style[4],
               zoom_start=12, 
               control_scale=True, 
               prefer_canvas=True)

# output filename
outfp = "paris_services.html"

# We create a feature group (layer) for each set of data
feature_group1 = folium.FeatureGroup(name='Autolib stations')
feature_group2 = folium.FeatureGroup(name='Parkings')
feature_group3 = folium.FeatureGroup(name='Gas stations')
feature_group4 = folium.FeatureGroup(name='Taxi stations')

# put autolib stations on the map with rough availability, and add to layer 1
dfa = pd.read_table(infpa, sep=';')
dfa.apply(lambda row:folium.CircleMarker(literal_eval(row['Geo point']),
                                        radius=4,
                                        fill=True,
                                        fill_opacity=0.6,
                                        color=colorMapa[plot_color_cars_remaining(row['Cars'])],
                                        popup='{}.\\n{} cars left'.format(row['Address'].replace("'","\\'"),row['Cars'])).add_to(feature_group1),
                                        axis=1)

# add parkings, and add to layer 2
dfp = pd.read_table(infpp, sep=';')
dfp.apply(lambda row:folium.Marker(literal_eval(row['geo_point_2d']),
                                        icon=folium.Icon(color='blue',prefix='fa',icon='product-hunt'),
                                        popup='{} - {}'.format(row['ADRESS_GEO'].replace("'","\\'"),row['SITE_DELEG'])).add_to(feature_group2),
                                        axis=1)

# add gas stations, and add to layer 3
dfc = pd.read_table(infpc, sep=',')
# only for the city of Paris
df2 = dfc.loc[dfc['commune']=='PARIS']
df2.apply(lambda row:folium.Marker(literal_eval(row['latlng']),
                                        icon=folium.Icon(color='lightgray',prefix='fa',icon='car'),
                                        popup='{}'.format(row['adresse'].replace("'","\\'"))).add_to(feature_group3),
                                        axis=1)

# add taxi stations, , and add to layer 4
dft = pd.read_table(infpt, sep=';')
dft.apply(lambda row:folium.Marker(literal_eval(row['geo_coordinates']),
                                        icon=folium.Icon(color='orange',prefix='fa',icon='taxi',icon_color='black'),
                                        popup='{}'.format(row['address'].replace("'","\\'"))).add_to(feature_group4),
                                        axis=1)

# add the layers to the map
feature_group1.add_to(m)
feature_group2.add_to(m)
feature_group3.add_to(m)
feature_group4.add_to(m)

# enable layer control
folium.LayerControl().add_to(m)

m.save(outfp)
m